In [2]:
from Smartis import *   
from Buttons import *   
from Chart2 import *
from ipywidgets import HBox, VBox, Layout
#from math import pi, sin, cos

class ElektronImFeld(Smartis):
    
    def __init__(self):
        super().__init__()
        print("KonstruktorE")
        #self.updateTime = 0.08
        
    def createButtons(self):
        self.buttonsObject = Buttons(self)
        
        button_play = self.buttonsObject.newTogglePlayPause("b_play", 40,40)
        button_reset = self.buttonsObject.newReset("b_reset", 40,40)
        
        
        slider_velocity = self.buttonsObject.newFloatSlider("s_velocity", 0,100,1)
        
        buttons_field = self.buttonsObject.newRadioButtons("b_field", ["E","B"], "Feldtyp") 
        
        slider_field = self.buttonsObject.newIntSlider("s_field", -10,10,0)
        
        
        self.buttonStatesDict = self.buttonsObject.getStates()
        self.oldButtonStatesDict = dict(self.buttonStatesDict)
        
            
    def createChart(self):

        # Ebenen definieren
        self.layers = ["background", "field", "path", "elektron"]
        self.chartWidth = 700
        self.chartHeight = 300
        self.chartObject = Chart(self.layers, self.chartWidth, self.chartHeight)
        
        
        ##Background
        self.x_fieldStart = 100
        self.y_fieldStart = 10        
        self.fieldWidth = 500
        self.fieldHeight = 280 
        currentLayer = "background"
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#c0c0c0")
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#ffffff")
        self.chartObject.changeLayerLook(currentLayer,"lineDash",[3,3])
        self.chartObject.drawRectangle(currentLayer,self.x_fieldStart,self.y_fieldStart,
            self.fieldWidth,self.fieldHeight,0)       
        self.chartObject.update(currentLayer) 
        
        self.createElectron()
        self.createPath()
        
            
        
    def createElectron(self):
        currentLayer = "elektron"
        
        self.chartObject.clearLayer(currentLayer)
        
        self.oldElectronPosition = [10,150,5]
        self.oldElectronVelocity = [0,0,0]
        
        self.chartObject.changeLayerLook(currentLayer,"fillingColor","#ffff00")
        self.chartObject.drawCircle(currentLayer,self.oldElectronPosition[0], self.oldElectronPosition[1],
            self.oldElectronPosition[2],0) 
        self.chartObject.update(currentLayer)
        
    def createPath(self):
        currentLayer = "path"
        self.chartObject.clearLayer(currentLayer)
        self.chartObject.changeLayerLook(currentLayer,"lineColor","#f0f0f0")
        self.chartObject.changeLayerLook(currentLayer,"lineWidth",2)
        self.chartObject.update(currentLayer)
        
        

        
    def updateField(self):
        
        currentLayer = "field"
        currentField = self.buttonStatesDict["b_field"]
        currentFieldStrength = int(self.buttonStatesDict["s_field"]*2)
        
        self.chartObject.clearLayer(currentLayer)
        
        if currentField == "B":
            self.chartObject.changeLayerLook(currentLayer,"lineColor","#00d000")
            self.chartObject.changeLayerLook(currentLayer,"fillingColor","#00d000")
        else:
            self.chartObject.changeLayerLook(currentLayer,"lineColor","#e8082c")
            self.chartObject.changeLayerLook(currentLayer,"fillingColor","#e8082c")
        
        if currentFieldStrength>= 0:
            self.chartObject.drawParralelArrows(currentLayer,self.x_fieldStart,self.y_fieldStart,
                self.fieldWidth, self.fieldHeight, currentFieldStrength, "vertical",10,10)
        else:
            self.chartObject.drawParralelArrows(currentLayer,self.x_fieldStart,
                self.y_fieldStart+self.fieldHeight,-self.fieldWidth, -self.fieldHeight, 
                currentFieldStrength, "vertical",10,10)
            
        self.chartObject.update(currentLayer)

            
    def updateElectron(self):
        currentElectronLayer = "elektron"
        currentPathLayer = "path"
        currentField = self.buttonStatesDict["b_field"]
        currentFieldStrength = self.buttonStatesDict["s_field"]
        sliderVelocity = self.buttonStatesDict["s_velocity"]
        oldXPosition = self.oldElectronPosition[0]
        oldYPosition = self.oldElectronPosition[1]
        oldZPosition = self.oldElectronPosition[2]
        oldXVelocity = self.oldElectronVelocity[0]
        oldYVelocity = self.oldElectronVelocity[1]
        oldZVelocity = self.oldElectronVelocity[2]
        t = 0.05
                
        if currentField == "E":
            newXPosition = float(oldXPosition) + t*sliderVelocity
            newZVelocity = oldZVelocity
            newZPosition = oldZPosition+oldZVelocity*t            
            
            if newXPosition > self.x_fieldStart and newXPosition <(self.x_fieldStart+self.fieldWidth):
                newYVelocity = oldYVelocity-currentFieldStrength*t
                newYPosition = float(oldYPosition) + (newYVelocity*t)
            else:
                newYVelocity = oldYVelocity
                newYPosition = oldYPosition+t*newYVelocity             


                
        elif currentField == "B":
            newXPosition = float(oldXPosition) + t*sliderVelocity
            newYVelocity = oldYVelocity
            newYPosition = oldYPosition+t*newYVelocity     
            
            if newXPosition > self.x_fieldStart and newXPosition <(self.x_fieldStart+self.fieldWidth):
                newZVelocity = oldZVelocity+t*sliderVelocity*currentFieldStrength/1000
                newZPosition = float(oldZPosition)+(newZVelocity*t)
            else:
                newZVelocity = oldZVelocity
                newZPosition = oldZPosition+oldZVelocity*t
            
            
        if (newXPosition > self.chartWidth or
            newYPosition < self.y_fieldStart or newYPosition > (self.y_fieldStart+self.fieldHeight) or
            newZPosition < 1):
                
            self.createElectron()
            self.buttonsObject.changeStates("b_play",False)
                
        else:
            
            self.chartObject.drawCircle(currentElectronLayer,newXPosition, newYPosition, newZPosition,0)  
            self.chartObject.drawCircle(currentPathLayer,newXPosition, newYPosition,1,1)
            self.oldElectronPosition[0] = newXPosition
            self.oldElectronPosition[1] = newYPosition
            self.oldElectronPosition[2] = newZPosition      
            self.oldElectronVelocity[1] = newYVelocity
            self.oldElectronVelocity[2] = newZVelocity
 
            self.chartObject.update(currentElectronLayer,currentPathLayer)
            

        

    def reset(self):        
        
        self.buttonsObject.changeStates("b_play",False)
        self.buttonsObject.changeStates("s_velocity",1)
        self.buttonsObject.changeStates("s_field",0)
        self.buttonsObject.changeStates("b_field","E")
                
        self.createElectron()
        self.createPath()
        self.updateField()
        self.buttonsObject.changeStates("b_reset",False)
        
        
    def update(self):     
                
        if not self.buttonStatesDict["b_reset"]:
            if self.oldButtonStatesDict["b_field"] != self.buttonStatesDict["b_field"]:
                self.updateField()
                #self.oldButtonStatesDict["b_field"] = self.buttonStatesDict["b_field"]

            if self.oldButtonStatesDict["s_field"] != self.buttonStatesDict["s_field"]:
                self.updateField()
                #self.oldButtonStatesDict["s_field"] = self.buttonStatesDict["s_field"]                                                            

            if self.buttonStatesDict["b_play"]:
                self.updateElectron()
        else:
            self.reset()

    def visualize(self): 
        self.buttonsDict = self.buttonsObject.getButtons()
        self.chart = self.chartObject.getChart()
      
        display(HBox([self.chart,self.buttonsDict["b_play"],self.buttonsDict["b_reset"]]))
        display(HBox([self.buttonsDict["b_field"],VBox([self.buttonsDict["s_velocity"],self.buttonsDict["s_field"]])]))


        


In [3]:
e = ElektronImFeld()
e.visualize()
e.start()


KonstruktorS
Konstruktor
KonstruktorE


1234 ElektronImFeld 08/01/2021, 12:26:30.613613 b_play True
1234 ElektronImFeld 08/01/2021, 12:26:33.020362 s_velocity 12.3
1234 ElektronImFeld 08/01/2021, 12:26:33.082860 s_velocity 13.1
 ElektronImFeld 08/01/2021, 12:26:33.267793 s_velocity 17.1
1234 ElektronImFeld 08/01/2021, 12:26:33.321178 s_velocity 19.5
1234 ElektronImFeld 08/01/2021, 12:26:33.383677 s_velocity 25.2
1234 ElektronImFeld 08/01/2021, 12:26:33.452690 s_velocity 30.8
1234 ElektronImFeld 08/01/2021, 12:26:33.521723 s_velocity 33.2
1234 ElektronImFeld 08/01/2021, 12:26:36.244403 b_play False
1234 ElektronImFeld 08/01/2021, 12:26:36.761567 b_reset True


In [3]:
e.__del__()

deleteS


In [4]:
e.buttonStatesDict

{'b_play': False,
 'b_reset': False,
 's_velocity': 1.0,
 'b_field': 'E',
 's_field': 0}

In [6]:
e.oldButtonStatesDict

{'b_play': False,
 'b_reset': False,
 's_velocity': 1.0,
 'b_field': 'E',
 's_field': 0}